In [1]:
# Donchian Weekly Trend Following System
import numpy as np
import pandas as pd

In [2]:
# Function definitions.
def donchian(prices, period):
    '''Calculate upper, lower, & middle Donchian lines.'''
    df = pd.DataFrame(prices.copy())
    df['upr'] = df.high.rolling(period).max().shift(periods=1)
    df['lwr'] = df.low.rolling(period).min().shift(periods=1)
    df['mid'] = 0.5 * (df.upr + df.lwr)
    return df


def remainder_zero(series, divisor):
    '''Increment number until remainder is zero.'''
    def increment_dividend(dividend, divisor):
        '''Increment dividend while remainder does not equal zero.'''
        while dividend % divisor != 0:
            dividend += 1
        return dividend
    series = series.map(lambda x: increment_dividend(x, divisor))
    return series


def style_table(trade_list):
    '''Format trade list.'''
    return (
        trade_list.style
            .format({
                'date': '{:%Y-%m-%d}',
                'open': '{:.3f}',
                'high': '{:.3f}',
                'low': '{:.3f}',
                'close': '{:.3f}',
                'upr': '{:.3f}',
                'lwr': '{:.3f}',
                'mid': '{:.3f}',
                'volatility': '{:.1%}',
                'risk_raw': '{:,.2f}',
                'risk': '{:,.2f}',
                'charges': '{:,.2f}',
                'sduty': '{:,.2f}',
                'cost': '{:,.2f}',
                'value': '{:,.2f}',
                'profit': '{:,.2f}',
                'pct': '{:,.1%}',
                'annual': '{:,.1%}',
                'cum_profit': '{:,.2f}',
            })
            .hide()
    )


def weekly(exchange, tidm):
    '''Generate weekly prices from SharePad csv file of daily prices.'''
    df = pd.read_csv(
        f'{exchange}_{tidm}_prices.csv',
        header=0,
        names=['date', 'open', 'high', 'low', 'close'],
        index_col=0,
        usecols=[0, 1, 2, 3, 4],
        parse_dates=True,
        dayfirst=True,
    )
    df = df.sort_index()
    functions = dict(open='first', high='max', low='min', close='last')
    df = df.resample('W-FRI').agg(functions)
    df = df / 100
    df.insert(0, 'tidm', tidm)
    return df

In [3]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
periods = [48, 24, 12, 6]  # System look back periods.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2  # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [4]:
# Import weekly prices.
prices = weekly(exchange, tidm)

In [5]:
# Trade signals.
signals = []
for sys, period in enumerate(periods):
    
    # Donchian channel.
    dc = donchian(prices, period)
    dc['sys'] = sys
    dc['period'] = period
    
    # Raw entry & exit signals.
    if sys == 0:
        dc['entry'] = np.where(dc.close > dc.upr, 1, 0)
    else:
        dc['entry'] = buy
    dc['exit'] = np.where(dc.close < dc.mid, 1, 0)
    
    # State variable.
    dc['state'] = 0
    for i in range(period, len(dc)):
        if dc.loc[dc.index[i], 'entry'] == 1 and dc.loc[dc.index[i - 1], 'state'] == 0:
            dc.loc[dc.index[i], 'state'] = 1
        elif dc.loc[dc.index[i], 'exit'] == 1:
            dc.loc[dc.index[i], 'state'] = 0
        else:
            dc.loc[dc.index[i], 'state'] = dc.loc[dc.index[i - 1], 'state']
            
    # Buy & sell signals.
    dc['buy'] = np.where(np.logical_and(dc.state == 1, dc.state.shift(periods=1) == 0), 1, 0)
    buy = dc.buy
    dc['sell'] = np.where(np.logical_and(dc.state == 0, dc.state.shift(periods=1) == 1), 1, 0)
    
    # Filter signals.
    if sys == 0:
        dc = pd.concat([dc[dc.buy == 1] , dc[dc.sell == 1]], axis=0)
    else:
        dc = dc[dc.sell == 1]
    signals.append(dc)

In [6]:
# Trade list indexed by date.
td = pd.concat(signals)
td = td.sort_index()

In [7]:
# Position size (buy side).
td['volatility'] = np.where(td.buy == 1, abs((td.mid - td.close) / td.close), 0)
td['risk_raw'] = np.where(td.buy == 1, ((position_size * risk_pct) / td.volatility), 0)
td['shares_raw'] = np.where(td.buy == 1, (td.risk_raw / td.close).astype('int'), 0)
td['shares'] = remainder_zero(td.shares_raw, len(periods)) # Adjust shares to be divisible by number of systems.
td['risk'] = td.close * td.shares # Adjust risk amount for revised share count.

# Position size (sell side).
for index, row in td.iterrows():
    if row['buy'] == 1:
        shares = row['shares']
    else:
        td.at[index, 'shares'] = int(shares / 4)

In [8]:
# Charges.
td['charges'] = td.index.values
td.charges = td.charges.shift()
td.charges = np.where(td.index == td.charges.values, 0, commission)

# Stamp duty.
td['sduty'] = np.where(td.buy==1, ((sduty / 100) * td.close * td.shares), 0)

# Cost to buy.
td['cost'] = np.where(td.buy == 1, (td.risk + td.charges + td.sduty), 0)

# Cost to sell.
for index, row in td.iterrows():
    if row['buy'] == 1:
        cost = row['cost']
    else:
        td.at[index, 'cost'] = (cost / 4)

In [9]:
# Returns.
td['value'] = np.where(td.sell == 1, ((td.close * td.shares) - td.charges), 0)
td['profit'] = np.where(td.sell == 1, td.value - td.cost, 0)
td['pct'] = np.where(td.sell == 1, (td.profit / td.cost), 0)

# Trade duration.
td['days'] = 0
for index, row in td.iterrows():
    if row['entry'] == 1:
        start_date = index
    else:
        td.at[index, 'days'] = index - start_date
td.days = td.days.astype('timedelta64[D]')
td.days = td.days.dt.days

# Annual percentage return.
td['annual'] = ((np.power(1 + td.profit / td.cost, (365 / td.days)) - 1))

# Cumulative profit.
td['cum_profit'] = td.profit.cumsum()

In [10]:
# Trade list indexed by trade.
td = td.reset_index()
td.insert(0, 'trade', td.state.cumsum())
style_table(td)

trade,date,tidm,open,high,low,close,upr,lwr,mid,sys,period,entry,exit,state,buy,sell,volatility,risk_raw,shares_raw,shares,risk,charges,sduty,cost,value,profit,pct,days,annual,cum_profit
1,2001-12-07,HSV,1.195,1.228,1.195,1.228,1.206,0.878,1.042,0,48,1,0,1,1,0,15.1%,"9,913.87",8075,8076,"9,914.91",11.95,49.57,"9,976.43",0.00,0.00,0.0%,0,0.0%,0.00
1,2002-02-01,HSV,1.346,1.346,1.303,1.308,1.359,1.271,1.315,3,6,0,1,0,0,1,0.0%,0.00,0,2019,0.00,11.95,0.00,"2,494.11","2,629.91",135.80,5.4%,56,41.3%,135.80
1,2002-02-22,HSV,1.308,1.308,1.257,1.257,1.359,1.190,1.275,2,12,0,1,0,0,1,0.0%,0.00,0,2019,0.00,11.95,0.00,"2,494.11","2,525.53",31.42,1.3%,77,6.1%,167.23
1,2002-04-05,HSV,1.238,1.249,1.209,1.215,1.359,1.104,1.231,1,24,0,1,0,0,1,0.0%,0.00,0,2019,0.00,11.95,0.00,"2,494.11","2,440.73",-53.38,-2.1%,119,-6.4%,113.85
1,2002-09-06,HSV,1.179,1.185,1.141,1.147,1.359,0.991,1.175,0,48,0,1,0,0,1,0.0%,0.00,0,2019,0.00,11.95,0.00,"2,494.11","2,303.64",-190.47,-7.6%,273,-10.1%,-76.62
2,2003-09-05,HSV,1.125,1.174,1.120,1.171,1.141,0.786,0.964,0,48,1,0,1,1,0,17.7%,"8,491.84",7254,7256,"8,493.87",11.95,42.47,"8,548.29",0.00,0.00,0.0%,0,0.0%,-76.62
2,2003-10-03,HSV,1.158,1.163,1.152,1.158,1.195,1.120,1.158,3,6,0,1,0,0,1,0.0%,0.00,0,1814,0.00,11.95,0.00,"2,137.07","2,088.12",-48.96,-2.3%,28,-26.1%,-125.57
2,2003-10-10,HSV,1.155,1.155,1.099,1.109,1.195,1.053,1.124,2,12,0,1,0,0,1,0.0%,0.00,0,1814,0.00,11.95,0.00,"2,137.07","2,000.14",-136.93,-6.4%,35,-49.9%,-262.51
2,2004-02-06,HSV,1.183,1.183,1.146,1.146,1.238,1.093,1.166,1,24,0,1,0,0,1,0.0%,0.00,0,1814,0.00,11.95,0.00,"2,137.07","2,066.53",-70.54,-3.3%,154,-7.6%,-333.05
2,2007-07-27,HSV,3.877,3.941,3.567,3.597,4.392,3.416,3.904,0,48,0,1,0,0,1,0.0%,0.00,0,1814,0.00,11.95,0.00,"2,137.07","6,512.83","4,375.75",204.8%,1421,33.1%,"4,042.70"
